## Modules

In [1]:
from keras import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.models import save_model
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
import json
import random
import string
'''
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')
'''

'\nnltk.download("punkt")\nnltk.download("wordnet")\nnltk.download(\'omw-1.4\')\n'

## Dataset

In [2]:
data = json.loads(open("dataset/intents.json").read())

## NLP

In [3]:
# initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()
# Each list to create
words = []
classes = []
doc_X = []
doc_y = []
# Loop through all the intents
# tokenize each pattern and append tokens to words, the patterns and
# the associated tag to their associated list
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    # add the tag to the classes if it's not there already 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])
# lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))

In [4]:
# list for training data
training = []
out_empty = [0] * len(classes)
# creating the bag of words model
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # mark the index of class that the current pattern is associated
    # to
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1
    # add the one hot encoded BoW and associated classes to training 
    training.append([bow, output_row])
# shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

# Deep Learning

In [5]:
# defining some parameters
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
# the deep learning model
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

2022-10-24 18:05:21.066655: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-24 18:05:21.068475: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               16768     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 28)                1820      
                                                                 
Total params: 26,844
Trainable params: 26,844
Non-trainable params: 0
__________________________________________

2022-10-24 18:05:21.589888: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-10-24 18:05:22.722851: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 2s 40ms/step - loss: 3.3179 - accuracy: 0.0500
Epoch 2/200
4/4 [==============================] - 0s 16ms/step - loss: 3.0432 - accuracy: 0.1417
Epoch 3/200
4/4 [==============================] - 0s 15ms/step - loss: 2.8105 - accuracy: 0.2333
Epoch 4/200
4/4 [==============================] - 0s 13ms/step - loss: 2.4078 - accuracy: 0.3333
Epoch 5/200
4/4 [==============================] - 0s 12ms/step - loss: 2.0359 - accuracy: 0.4750
Epoch 6/200
4/4 [==============================] - 0s 12ms/step - loss: 1.6949 - accuracy: 0.5917
Epoch 7/200
4/4 [==============================] - 0s 11ms/step - loss: 1.3749 - accuracy: 0.6417
Epoch 8/200
4/4 [==============================] - 0s 12ms/step - loss: 1.1833 - accuracy: 0.6750
Epoch 9/200
4/4 [==============================] - 0s 17ms/step - loss: 0.9995 - accuracy: 0.7417
Epoch 10/200
4/4 [==============================] - 0s 11ms/step - loss: 0.7572 - accuracy: 0.7917
Epoch 11/200
4/4 [=============

# functions

In [6]:
def clean_text(text): 
  tokens = word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab): 
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens: 
    for idx, word in enumerate(vocab):
      if word == w: 
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels): 
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0]
  thresh = 0.2
  y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]

  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list

def get_response(intents_list, intents_json): 
  tag = intents_list[0]
  list_of_intents = intents_json["intents"]
  for i in list_of_intents: 
    if i["tag"] == tag:
      result = random.choice(i["responses"])
      break
  return result

In [7]:
save_model(model, "chatbot_model.h5", save_format="h5")